# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 12 2022 10:39AM,248441,10,8632750-BO,MedStone Pharma LLC,Released
1,Oct 12 2022 10:37AM,248440,19,ADV80006435,"AdvaGen Pharma, Ltd",Released
2,Oct 12 2022 10:37AM,248440,19,ADV80006442,"AdvaGen Pharma, Ltd",Released
3,Oct 12 2022 10:37AM,248440,19,ADV80006443,"AdvaGen Pharma, Ltd",Released
4,Oct 12 2022 10:36AM,248439,10,8637005,"Snap Medical Industries, LLC",Released
5,Oct 12 2022 10:33AM,248438,10,Enova-10800,Emerginnova,Released
6,Oct 12 2022 10:23AM,248435,19,8637010,"Graystone, LLC",Released
7,Oct 12 2022 10:23AM,248435,19,8637016,"Graystone, LLC",Released
8,Oct 12 2022 10:21AM,248437,19,8637011,"Graystone, LLC",Released
9,Oct 12 2022 10:17AM,248434,10,8326064-BO,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,248437,Released,1
26,248438,Released,1
27,248439,Released,1
28,248440,Released,3
29,248441,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248437,NaN,NaN,1.0
248438,NaN,NaN,1.0
248439,NaN,NaN,1.0
248440,NaN,NaN,3.0
248441,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248080,0.0,0.0,1.0
248265,0.0,0.0,30.0
248347,0.0,0.0,1.0
248350,0.0,1.0,0.0
248378,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248080,0,0,1
248265,0,0,30
248347,0,0,1
248350,0,1,0
248378,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248080,0,0,1
1,248265,0,0,30
2,248347,0,0,1
3,248350,0,1,0
4,248378,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248080,,,1
1,248265,,,30
2,248347,,,1
3,248350,,1,
4,248378,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 12 2022 10:39AM,248441,10,MedStone Pharma LLC
1,Oct 12 2022 10:37AM,248440,19,"AdvaGen Pharma, Ltd"
4,Oct 12 2022 10:36AM,248439,10,"Snap Medical Industries, LLC"
5,Oct 12 2022 10:33AM,248438,10,Emerginnova
6,Oct 12 2022 10:23AM,248435,19,"Graystone, LLC"
8,Oct 12 2022 10:21AM,248437,19,"Graystone, LLC"
9,Oct 12 2022 10:17AM,248434,10,"Citieffe, Inc."
10,Oct 12 2022 9:56AM,248432,12,Hush Hush
11,Oct 12 2022 9:44AM,248265,15,"Person & Covey, Inc."
41,Oct 12 2022 9:39AM,248430,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 12 2022 10:39AM,248441,10,MedStone Pharma LLC,,,1
1,Oct 12 2022 10:37AM,248440,19,"AdvaGen Pharma, Ltd",,,3
2,Oct 12 2022 10:36AM,248439,10,"Snap Medical Industries, LLC",,,1
3,Oct 12 2022 10:33AM,248438,10,Emerginnova,,,1
4,Oct 12 2022 10:23AM,248435,19,"Graystone, LLC",,,2
5,Oct 12 2022 10:21AM,248437,19,"Graystone, LLC",,,1
6,Oct 12 2022 10:17AM,248434,10,"Citieffe, Inc.",,,1
7,Oct 12 2022 9:56AM,248432,12,Hush Hush,,1,
8,Oct 12 2022 9:44AM,248265,15,"Person & Covey, Inc.",,,30
9,Oct 12 2022 9:39AM,248430,15,"Mizner Bioscience, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 10:39AM,248441,10,MedStone Pharma LLC,1,,
1,Oct 12 2022 10:37AM,248440,19,"AdvaGen Pharma, Ltd",3,,
2,Oct 12 2022 10:36AM,248439,10,"Snap Medical Industries, LLC",1,,
3,Oct 12 2022 10:33AM,248438,10,Emerginnova,1,,
4,Oct 12 2022 10:23AM,248435,19,"Graystone, LLC",2,,
5,Oct 12 2022 10:21AM,248437,19,"Graystone, LLC",1,,
6,Oct 12 2022 10:17AM,248434,10,"Citieffe, Inc.",1,,
7,Oct 12 2022 9:56AM,248432,12,Hush Hush,,1,
8,Oct 12 2022 9:44AM,248265,15,"Person & Covey, Inc.",30,,
9,Oct 12 2022 9:39AM,248430,15,"Mizner Bioscience, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 10:39AM,248441,10,MedStone Pharma LLC,1,,
1,Oct 12 2022 10:37AM,248440,19,"AdvaGen Pharma, Ltd",3,,
2,Oct 12 2022 10:36AM,248439,10,"Snap Medical Industries, LLC",1,,
3,Oct 12 2022 10:33AM,248438,10,Emerginnova,1,,
4,Oct 12 2022 10:23AM,248435,19,"Graystone, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 10:39AM,248441,10,MedStone Pharma LLC,1.0,NaN,NaN
1,Oct 12 2022 10:37AM,248440,19,"AdvaGen Pharma, Ltd",3.0,NaN,NaN
2,Oct 12 2022 10:36AM,248439,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
3,Oct 12 2022 10:33AM,248438,10,Emerginnova,1.0,NaN,NaN
4,Oct 12 2022 10:23AM,248435,19,"Graystone, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 10:39AM,248441,10,MedStone Pharma LLC,1.0,0.0,0.0
1,Oct 12 2022 10:37AM,248440,19,"AdvaGen Pharma, Ltd",3.0,0.0,0.0
2,Oct 12 2022 10:36AM,248439,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
3,Oct 12 2022 10:33AM,248438,10,Emerginnova,1.0,0.0,0.0
4,Oct 12 2022 10:23AM,248435,19,"Graystone, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1987402,24.0,20.0,8.0
12,496810,0.0,2.0,0.0
15,993548,62.0,37.0,0.0
16,496776,2.0,0.0,0.0
19,1242062,6.0,2.0,0.0
21,1241661,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1987402,24.0,20.0,8.0
1,12,496810,0.0,2.0,0.0
2,15,993548,62.0,37.0,0.0
3,16,496776,2.0,0.0,0.0
4,19,1242062,6.0,2.0,0.0
5,21,1241661,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,24.0,20.0,8.0
1,12,0.0,2.0,0.0
2,15,62.0,37.0,0.0
3,16,2.0,0.0,0.0
4,19,6.0,2.0,0.0
5,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,12,Released,0.0
2,15,Released,62.0
3,16,Released,2.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,8.0,0.0,0.0,0.0,0.0,0.0
Executing,20.0,2.0,37.0,0.0,2.0,1.0
Released,24.0,0.0,62.0,2.0,6.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,2.0,37.0,0.0,2.0,1.0
2,Released,24.0,0.0,62.0,2.0,6.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,2.0,37.0,0.0,2.0,1.0
2,Released,24.0,0.0,62.0,2.0,6.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()